In [1]:
import duckdb
import pandas as pd
import os

In [3]:
base_path = '/home/fedepicado/'
modelos_path = base_path + 'buckets/b1/modelos/'
db_path = base_path + 'buckets/b1/db/'
datasets_path = base_path + 'buckets/b1/datasets/'
exp_path = base_path + 'buckets/b1/exp/'
dataset_file = 'competencia_03_crudo.csv'
full_path = datasets_path + dataset_file

# Verifica si el archivo existe
if os.path.exists(full_path):
    print("Archivo encontrado:", full_path)
else:
    print("Archivo no encontrado:", full_path)

Archivo encontrado: /home/fedepicado/buckets/b1/datasets/competencia_03_crudo.csv


In [4]:
conn= duckdb.connect(database=':memory:')

df= pd.read_csv(datasets_path + dataset_file)

In [7]:
conn.register('competencia_03_crudo', df)

In [9]:
conn.execute("""
create or replace table competencia_03 as
with periodos as (
    select distinct foto_mes from competencia_03_crudo
), clientes as (
    select distinct numero_de_cliente from competencia_03_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        c.*
        , if(c.numero_de_cliente is null, 0, 1) as mes_0 -- genero la columna mes_0 y sobre ella genero mes_1 y mes_2
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , if(mes_2 = 1, 'CONTINUA',
             if(mes_1 = 0, 'BAJA+1',
                if(mes_2 = 0,'BAJA+2',NULL))) as clase_ternaria -- Replazar null por la lógica que genera el target
    from todo t
    left join competencia_03_crudo c using (numero_de_cliente, foto_mes)
) 

select * EXCLUDE (mes_0, mes_1, mes_2) -- descarto las columnas auxiliares
from clase_ternaria
where mes_0 = 1 -- descarto las filas de los clientes para los meses previos a que ingresaran al banco.
""")

In [10]:
resultado = conn.execute("SELECT foto_mes, clase_ternaria FROM competencia_03").df()

In [13]:
conteo_clase_ternaria = resultado.groupby(['foto_mes', 'clase_ternaria']).size().reset_index(name='cantidad')
conteo_clase_ternaria.tail(60)

,foto_mes,clase_ternaria,cantidad
34,201912,BAJA+2,598
35,201912,CONTINUA,139329
36,202001,BAJA+1,605
37,202001,BAJA+2,502
38,202001,CONTINUA,142859
39,202002,BAJA+1,508
40,202002,BAJA+2,185
41,202002,CONTINUA,146416
42,202003,BAJA+1,186
43,202003,BAJA+2,378


In [12]:
file_path = f"{datasets_path}/competencia_03.parquet" 

In [14]:
conn.execute(f"COPY competencia_03 TO '{file_path}' (FORMAT 'parquet')")